Establish connection to database on SQL server on Ubuntu machine in Virtualbox

In [1]:
#!/usr/bin/env py36
import mysql.connector
from mysql.connector import Error

try:
    connection = mysql.connector.connect(
        host="192.168.56.101",
        port="3306",
        user="root",
        passwd="************",
        database="employees")

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL database... MySQL Server version on ", db_Info)

        # Creating 'cursor' object to execute database operations
        cursor = connection.cursor()
        query = "SELECT database();"
        cursor.execute(query)
        record = cursor.fetchone()
        print("Your connected to - ", record)

except Error as e:
    print ("Error while connecting to MySQL", e)

finally:
    # closing database connection.
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL database... MySQL Server version on  5.7.27-0ubuntu0.18.04.1
Your connected to -  ('employees',)
MySQL connection is closed


A function to execute simple_query:

In [2]:
#!/usr/bin/env py36
import mysql.connector
from pandas import DataFrame
from mysql.connector import Error

def simple_query(query):
    try:
        connection = mysql.connector.connect(
            host="192.168.56.101",
            port="3306",
            user="root",
            passwd="************",
            database="employees") # name of the whole database (may include sub-database with the same name)

        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL database... MySQL Server version on ", db_Info)

            # Creating 'cursor' object to execute database operations - queries
            cursor = connection.cursor()
            # query = "SELECT * from salaries limit 5;"
            cursor.execute(query)
            record = cursor.fetchall()
            # print under panda format
            df = DataFrame(record)
            # get all columns' name from record ['EmP_ID','Salary','from_date','to_date']
            df.columns = [i[0] for i in cursor.description]
            print('\n')
            print(df)
            print('\n')

    except Error as e:
        print ("Error while connecting to MySQL", e)

    finally:
        # closing database connection.
        if (connection.is_connected()):
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

Some other queries:

In [3]:
simple_query("SHOW tables;")

Connected to MySQL database... MySQL Server version on  5.7.27-0ubuntu0.18.04.1


    Tables_in_employees
0      current_dept_emp
1           departments
2              dept_emp
3  dept_emp_latest_date
4          dept_manager
5             employees
6              salaries
7                titles


MySQL connection is closed


In [4]:
# describe a table, ex: salaries
simple_query("DESCRIBE salaries;")

Connected to MySQL database... MySQL Server version on  5.7.27-0ubuntu0.18.04.1


       Field     Type Null  Key Default Extra
0     emp_no  int(11)   NO  PRI    None      
1     salary  int(11)   NO         None      
2  from_date     date   NO  PRI    None      
3    to_date     date   NO         None      


MySQL connection is closed


NULL - NO : mean the data is forced to be filled and NULL value is not allowed.

And in this case, it will be filled with a number.

PRI key: Primary key, is a key in a relational database that is unique for each record.

E.g. 2 people may have the same name, but their personal ID are different. Primary key acts like personal ID for this example.

In [5]:
# select all columns (keys) in side a tables, ex: salaries
simple_query("SELECT * from salaries limit 5;")

Connected to MySQL database... MySQL Server version on  5.7.27-0ubuntu0.18.04.1


   emp_no  salary   from_date     to_date
0   10009   60929  1985-02-18  1986-02-18
1   10013   40000  1985-10-20  1986-10-20
2   10048   40000  1985-02-24  1986-02-24
3   10064   40000  1985-11-20  1986-11-20
4   10070   55999  1985-10-14  1986-10-14


MySQL connection is closed


In [6]:
simple_query("SELECT * from employees limit 5;")

Connected to MySQL database... MySQL Server version on  5.7.27-0ubuntu0.18.04.1


   emp_no  birth_date first_name last_name gender   hire_date
0   10001  1953-09-02     Georgi   Facello      M  1986-06-26
1   10002  1964-06-02    Bezalel    Simmel      F  1985-11-21
2   10003  1959-12-03      Parto   Bamford      M  1986-08-28
3   10004  1954-05-01  Chirstian   Koblick      M  1986-12-01
4   10005  1955-01-21    Kyoichi  Maliniak      M  1989-09-12


MySQL connection is closed


In [7]:
simple_query("SELECT * from titles limit 5;")

Connected to MySQL database... MySQL Server version on  5.7.27-0ubuntu0.18.04.1


   emp_no               title   from_date     to_date
0   10009  Assistant Engineer  1985-02-18  1990-02-18
1   10013        Senior Staff  1985-10-20  9999-01-01
2   10048            Engineer  1985-02-24  1987-01-27
3   10064               Staff  1985-11-20  1992-03-02
4   10070    Technique Leader  1985-10-14  9999-01-01


MySQL connection is closed


In [8]:
simple_query("SELECT * from departments limit 5;")

Connected to MySQL database... MySQL Server version on  5.7.27-0ubuntu0.18.04.1


  dept_no         dept_name
0    d009  Customer Service
1    d005       Development
2    d002           Finance
3    d003   Human Resources
4    d001         Marketing


MySQL connection is closed


In [9]:
# Join tables employees and salaries
simple_query("SELECT * from employees e join salaries s on e.emp_no=s.emp_no limit 5;")

Connected to MySQL database... MySQL Server version on  5.7.27-0ubuntu0.18.04.1


   emp_no  birth_date first_name last_name gender   hire_date  emp_no  salary  \
0   10001  1953-09-02     Georgi   Facello      M  1986-06-26   10001   60117   
1   10001  1953-09-02     Georgi   Facello      M  1986-06-26   10001   62102   
2   10001  1953-09-02     Georgi   Facello      M  1986-06-26   10001   66074   
3   10001  1953-09-02     Georgi   Facello      M  1986-06-26   10001   66596   
4   10001  1953-09-02     Georgi   Facello      M  1986-06-26   10001   66961   

    from_date     to_date  
0  1986-06-26  1987-06-26  
1  1987-06-26  1988-06-25  
2  1988-06-25  1989-06-25  
3  1989-06-25  1990-06-25  
4  1990-06-25  1991-06-25  


MySQL connection is closed


Some columns are just repeated, we can select which columns to display:

In [10]:
# Join tables employees and salaries with selective columns
simple_query("SELECT e.emp_no, e.birth_date, e.first_name, e.last_name, e.gender, s.salary from employees e join salaries s on e.emp_no=s.emp_no limit 5;")

Connected to MySQL database... MySQL Server version on  5.7.27-0ubuntu0.18.04.1


   emp_no  birth_date first_name last_name gender  salary
0   10001  1953-09-02     Georgi   Facello      M   60117
1   10001  1953-09-02     Georgi   Facello      M   62102
2   10001  1953-09-02     Georgi   Facello      M   66074
3   10001  1953-09-02     Georgi   Facello      M   66596
4   10001  1953-09-02     Georgi   Facello      M   66961


MySQL connection is closed


In [11]:
#!/usr/bin/env py36
import mysql.connector
from mysql.connector import Error
from pandas import DataFrame

# select employes with specific hire and end dates
def period_query(query, start_date, end_date):
    try:
        connection = mysql.connector.connect(
            host="192.168.56.101",
            port="3306",
            user="root",
            passwd="************",
            database="employees") # name of the whole database (may include sub-database with the same name)

        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL database... MySQL Server version on ", db_Info)

            # Creating 'cursor' object to execute database operations - queries
            cursor = connection.cursor()
            cursor.execute(query, (start_date, end_date))
            record = cursor.fetchall()
            
            df = DataFrame(record)
            df.columns = [i[0] for i in cursor.description]
            print('\n')
            print(df)
            print('\n')

    except Error as e:
        print ("Error while connecting to MySQL", e)

    finally:
        # closing database connection.
        if (connection.is_connected()):
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [12]:
import datetime
# select employes hired within January 2000
hire_start = datetime.date(2000, 1, 1)
hire_end = datetime.date(2000, 1, 31)

query = ("SELECT * FROM employees "
        "WHERE hire_date BETWEEN %s AND %s limit 10;")
    
period_query(query, hire_start, hire_end)  

Connected to MySQL database... MySQL Server version on  5.7.27-0ubuntu0.18.04.1


   emp_no  birth_date   first_name   last_name gender   hire_date
0   47291  1960-09-09          Ulf      Flexer      M  2000-01-12
1   60134  1964-04-21        Seshu    Rathonyi      F  2000-01-02
2   72329  1953-02-09        Randi        Luit      F  2000-01-02
3  108201  1955-04-14  Mariangiola     Boreale      M  2000-01-01
4  205048  1960-09-12        Ennio      Alblas      F  2000-01-06
5  222965  1959-08-07      Volkmar       Perko      F  2000-01-13
6  226633  1958-06-10       Xuejun  Benzmuller      F  2000-01-04
7  227544  1954-11-17       Shahab     Demeyer      M  2000-01-08
8  422990  1953-04-09        Jaana    Verspoor      F  2000-01-11
9  424445  1953-04-27        Jeong     Boreale      M  2000-01-03


MySQL connection is closed
